<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Подготовка" data-toc-modified-id="Подготовка-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Подготовка</a></span><ul class="toc-item"><li><span><a href="#Подключение-библиотек" data-toc-modified-id="Подключение-библиотек-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Подключение библиотек</a></span></li><li><span><a href="#Регуляризация-комментариев" data-toc-modified-id="Регуляризация-комментариев-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Регуляризация комментариев</a></span></li><li><span><a href="#Лемматизация-комментариев" data-toc-modified-id="Лемматизация-комментариев-1.3"><span class="toc-item-num">1.3&nbsp;&nbsp;</span>Лемматизация комментариев</a></span></li></ul></li><li><span><a href="#Обучение" data-toc-modified-id="Обучение-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Обучение</a></span><ul class="toc-item"><li><span><a href="#Создание-выборок" data-toc-modified-id="Создание-выборок-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Создание выборок</a></span></li><li><span><a href="#Модель-логистической-регрессии" data-toc-modified-id="Модель-логистической-регрессии-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>Модель логистической регрессии</a></span></li><li><span><a href="#Модель-случайного-леса" data-toc-modified-id="Модель-случайного-леса-2.3"><span class="toc-item-num">2.3&nbsp;&nbsp;</span>Модель случайного леса</a></span></li><li><span><a href="#Модель-Catboost" data-toc-modified-id="Модель-Catboost-2.4"><span class="toc-item-num">2.4&nbsp;&nbsp;</span>Модель Catboost</a></span></li><li><span><a href="#Модель-LGBM" data-toc-modified-id="Модель-LGBM-2.5"><span class="toc-item-num">2.5&nbsp;&nbsp;</span>Модель LGBM</a></span></li><li><span><a href="#Проверка-лучшей-модели-на-тестовой-выборке" data-toc-modified-id="Проверка-лучшей-модели-на-тестовой-выборке-2.6"><span class="toc-item-num">2.6&nbsp;&nbsp;</span>Проверка лучшей модели на тестовой выборке</a></span></li></ul></li><li><span><a href="#Выводы" data-toc-modified-id="Выводы-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Выводы</a></span></li>

# Описание проекта

Необходимо классифицировать комментарии на токсичные и не токсичные. В распоряжении есть набор данных с разметкой о токсичности комментариев.

Задачи:
- подготовить данные
- обучить разные модели и выбрать лучшую (модель со значением метрики качества *F1* не меньше 0.75)

## Подготовка

### Подключение библиотек 

In [1]:
import pandas as pd
import numpy as np
import re
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression                    # импорт модели логистической регрессии
from sklearn.ensemble import RandomForestClassifier                    # импорт модели случайного леса           
from catboost import CatBoostClassifier                                # импорт модели CatBoost
from lightgbm import LGBMClassifier                                    # импорт модели LGBM
from sklearn.feature_extraction.text import TfidfVectorizer
import nltk
from nltk.stem import WordNetLemmatizer 
nltk.download('wordnet')
nltk.download('punkt')
from nltk.corpus import stopwords as nltk_stopwords
from sklearn.metrics import f1_score
import warnings
warnings.simplefilter("ignore")                                         # игнорируем предупреждения

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Миша\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Миша\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [3]:
data = pd.read_csv('...')                                # чтение файла

In [4]:
data.info()                                                             # вывод информации о датафрейме

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159571 entries, 0 to 159570
Data columns (total 2 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   text    159571 non-null  object
 1   toxic   159571 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 2.4+ MB


In [5]:
data.head(15)                                                           # вывод первых 15 строк датафрейма

,text,toxic
0,Explanation\nWhy the edits made under my usern...,0
1,D'aww! He matches this background colour I'm s...,0
2,"Hey man, I'm really not trying to edit war. It...",0
3,"""\nMore\nI can't make any real suggestions on ...",0
4,"You, sir, are my hero. Any chance you remember...",0
5,"""\n\nCongratulations from me as well, use the ...",0
6,COCKSUCKER BEFORE YOU PISS AROUND ON MY WORK,1
7,Your vandalism to the Matt Shirvington article...,0
8,Sorry if the word 'nonsense' was offensive to ...,0
9,alignment on this subject and which are contra...,0


**Вывод**

Пропуски не обнаружены, типы данных подходящие. Язык, на котором оставляют комментарии оказался английским, запомним это.

Кроме того, есть подозрение несбалансированности классов токсичных и нетоксичных комментариев. Проверим это.

In [6]:
print('Количество токсичных комментариев:', data[data.toxic == 1]['toxic'].count())
print('Количество нетоксичных комментариев:', data[data.toxic == 0]['toxic'].count())

Количество токсичных комментариев: 16225
Количество нетоксичных комментариев: 143346


**Вывод**

Действительно размеры классов отличаются, это необходимо учитывать при обучении некоторых моделей, которые чувствительны к дисбалансу.

### Регуляризация комментариев

In [7]:
data['text'] = data['text'].apply(lambda x: re.sub(r'[^a-zA-Z ]', ' ', x))    # замена ненужных символов на пробелы
data['text'] = data['text'].apply(lambda x: x.lower())                        # приведение текста к нижнему регистру

data['text'] = data['text'].apply(lambda x: x.split())                        # создание списка без множества пробелов
data['text'] = data['text'].apply(lambda x: ' '.join(x))                      # сшивка комметариев
data.head()                                                                   # проверка результата  

,text,toxic
0,explanation why the edits made under my userna...,0
1,d aww he matches this background colour i m se...,0
2,hey man i m really not trying to edit war it s...,0
3,more i can t make any real suggestions on impr...,0
4,you sir are my hero any chance you remember wh...,0


**Вывод**

Ненужные символы отсутствуют.

### Лемматизация комментариев

In [8]:
lmtzr = WordNetLemmatizer()                                                  # вызов лемматизатора
data['text'] = data['text'].apply(lambda x: nltk.word_tokenize(x))           # токенизация комментариев


In [9]:
data['text'] = data['text'].apply(                                           # лемматизация комментариев
                    lambda lst:[lmtzr.lemmatize(word) for word in lst])      #

In [10]:
data.head(15)                                                                # проверка результата                     

,text,toxic
0,"[explanation, why, the, edits, made, under, my...",0
1,"[d, aww, he, match, this, background, colour, ...",0
2,"[hey, man, i, m, really, not, trying, to, edit...",0
3,"[more, i, can, t, make, any, real, suggestion,...",0
4,"[you, sir, are, my, hero, any, chance, you, re...",0
5,"[congratulation, from, me, a, well, use, the, ...",0
6,"[cocksucker, before, you, piss, around, on, my...",1
7,"[your, vandalism, to, the, matt, shirvington, ...",0
8,"[sorry, if, the, word, nonsense, wa, offensive...",0
9,"[alignment, on, this, subject, and, which, are...",0


In [11]:
data['text'] = data['text'].apply(lambda x: ' '.join(x))         # склейка комментариев

In [12]:
data.head(15)                                                    # проверка результата

,text,toxic
0,explanation why the edits made under my userna...,0
1,d aww he match this background colour i m seem...,0
2,hey man i m really not trying to edit war it s...,0
3,more i can t make any real suggestion on impro...,0
4,you sir are my hero any chance you remember wh...,0
5,congratulation from me a well use the tool wel...,0
6,cocksucker before you piss around on my work,1
7,your vandalism to the matt shirvington article...,0
8,sorry if the word nonsense wa offensive to you...,0
9,alignment on this subject and which are contra...,0


**Вывод**

Комментарии лемматизированы. Об этом свидетельсвует, например, комментарий с индексом 11. В нём присутствует слово 'let', которое в исходном нелемматизированном виды было 'lets'.

## Обучение

### Создание выборок

In [13]:
train_valid, test = train_test_split(data, test_size = 0.1, random_state = 12345)      # в тестовую выборку идёт 10% данных
train, valid = train_test_split(train_valid, test_size = 0.111, random_state = 12345)  # в валидационную выборку идёт 10% данных
print(f'Размер обучающей выборки: {train.shape[0]}')
print(f'Размер валидационной выборки: {valid.shape[0]}')
print(f'Размер тестовой выборки: {test.shape[0]}')

Размер обучающей выборки: 127671
Размер валидационной выборки: 15942
Размер тестовой выборки: 15958


In [14]:
corpus_train = train['text'].values.astype('U')                  # корпус обучающей выборки
corpus_valid = valid['text'].values.astype('U')                  # корпус валидационной выборки
corpus_test = test['text'].values.astype('U')                    # корпус тестовой выборки

corpus_train_valid = train_valid['text'].values.astype('U')      # корпус выборки обучающая+валидационная

In [16]:
nltk.download('stopwords')
stopwords = set(nltk_stopwords.words('english'))                 # в качестве стоп-слов выбираем английские
count_tf_idf = TfidfVectorizer(stop_words=stopwords)             # передаём счётчику TF-IDF стоп-слова



[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Миша\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [17]:
tf_idf_train = count_tf_idf.fit_transform(corpus_train)              # обучение и расчёт TF-IDF для корпуса обучающей выборки
tf_idf_valid = count_tf_idf.transform(corpus_valid)                  # расчёт TF-IDF для корпуса валидационной выборки          
tf_idf_test = count_tf_idf.transform(corpus_test)                    # расчёт TF-IDF для корпуса тестовой выборки
tf_idf_train_valid = count_tf_idf.transform(corpus_train_valid)      # расчёт TF-IDF для корпуса выборки обучающая+валидационная

In [ ]:
#КОД РЕВЬЮЕРА
# tf_idf_train = count_tf_idf.fit_transform(corpus_train)           
# tf_idf_valid = count_tf_idf.transform(corpus_valid)                         
# tf_idf_test = count_tf_idf.transform(corpus_test)               
# tf_idf_train_valid = count_tf_idf.transform(corpus_train_valid) 


### Модель логистической регрессии

In [18]:
features_train = tf_idf_train
target_train = train['toxic']

features_valid = tf_idf_valid
target_valid = valid['toxic']

model = LogisticRegression(class_weight='balanced')   # модель логистической регрессии с учётом разных весов двух классов 

model.fit(features_train, target_train)

predictions_1 = model.predict(features_valid)

print(f'Метрика F1: {f1_score(target_valid, predictions_1)}')

Метрика F1: 0.7451923076923077


### Модель случайного леса

In [34]:
model_forest = RandomForestClassifier(random_state=12345, n_estimators=5, class_weight='balanced')  # модель случайного леса с
# количеством деревьев 5 и учётом разных весов двух классов 

model_forest.fit(features_train, target_train)

predictions_1 = model_forest.predict(features_valid)

print(f'Метрика F1: {f1_score(target_valid, predictions_1)}')

Метрика F1: 0.6107644305772231


### Модель Catboost

In [35]:
model = CatBoostClassifier(loss_function="Logloss", iterations=60, learning_rate = 0.8, max_depth = 8) # модель градиентого
# бустинга Catboost с 60 итерациями, скоростью обучения 0.8 и максимальной глубиной дерева 8
model.fit(features_train, target_train, verbose=1)                          # обучение модели Catboost на обучающей выборке

predictions_1 = model.predict(features_valid)                               # построение предсказаний

print(f'Метрика F1 составила: {f1_score(target_valid, predictions_1)}')

0:	learn: 0.2753481	total: 11.5s	remaining: 11m 19s
1:	learn: 0.2311965	total: 21.6s	remaining: 10m 26s
2:	learn: 0.2129660	total: 33.5s	remaining: 10m 36s
3:	learn: 0.1982042	total: 44.5s	remaining: 10m 22s
4:	learn: 0.1892004	total: 56.6s	remaining: 10m 22s
5:	learn: 0.1833688	total: 1m 7s	remaining: 10m 5s
6:	learn: 0.1777840	total: 1m 17s	remaining: 9m 44s
7:	learn: 0.1714628	total: 1m 29s	remaining: 9m 44s
8:	learn: 0.1674961	total: 1m 42s	remaining: 9m 39s
9:	learn: 0.1647220	total: 1m 52s	remaining: 9m 24s
10:	learn: 0.1607686	total: 2m 4s	remaining: 9m 14s
11:	learn: 0.1583151	total: 2m 14s	remaining: 8m 59s
12:	learn: 0.1551148	total: 2m 25s	remaining: 8m 46s
13:	learn: 0.1530476	total: 2m 36s	remaining: 8m 35s
14:	learn: 0.1508345	total: 2m 43s	remaining: 8m 9s
15:	learn: 0.1485123	total: 2m 53s	remaining: 7m 56s
16:	learn: 0.1462731	total: 3m 1s	remaining: 7m 38s
17:	learn: 0.1445631	total: 3m 8s	remaining: 7m 20s
18:	learn: 0.1427651	total: 3m 14s	remaining: 7m
19:	learn: 0

### Модель LGBM

In [19]:
model = LGBMClassifier(n_estimators = 200)                       # модель LGBM с увеличенным количеством деревьев до 200

model.fit(features_train, target_train)                          # обучение модели LGBM на обучающей выборке

predictions_1 = model.predict(features_valid)                               # построение предсказаний

print(f'Метрика F1 составила: {f1_score(target_valid, predictions_1)}')

Метрика F1 составила: 0.7777003484320557


**Вывод**

Лучшую метрику показала модель LGBM с увеличенным количеством деревьев до 200. 

### Проверка лучшей модели на тестовой выборке

In [20]:
features_test = tf_idf_test                                               # признаки тестовой выборки
target_test = test['toxic']                                               # целевой признак тестовой выборки

features_train_valid = tf_idf_train_valid                                 # признаки выборки обучающая+валидационная
target_train_valid = train_valid['toxic']                                 # целевой признак быборки обучающая+валидационная

model.fit(features_train_valid, target_train_valid)                       # обучение лучшей модели 

predictions_2 = model.predict(features_test)                              # построение предсказаний

print(f'Метрика F1 составила: {f1_score(target_test, predictions_2)}')    # вычисление метрики F1


Метрика F1 составила: 0.7584789311408017


## Выводы

В ходе разработки метода оценки комментариев было выяснено, что комментарии на английском языке. Датафрейм с комментариями и правильной разметкой не содержал пропусков или неверных типов данных. Был обнаружен дисбаланс классов, что в дальнейшем учитывалось при построении моделей.

Подготовка данных заключалась в регуляризации и лемматизации комментариев. Выделение признаков производилось с помощью вычисления TF-IDF. Счётчику были также переданы стоп-слова, которые были исключены из расчёта. После подготовки признаков для всех необходимых выборок были рассмотрены разные модели МО.
- Логистическая регрессия с учётом дисбаланса: F1 = 0.75
- Случайный лес с количеством деревьев 5 и учётом разных весов двух классов: F1 = 0.61

Использование большого числа решающих деревьев приводит к долгому обучению, поэтому были рассмотрены модели град. бустинга
Модели градиентного бустинга:
- Catboost c 60 итерациями, скоростью обучения 0.8 и максимальной глубиной дерева 8: F1 = 0.75
- LGBM c количеством деревьев 200: 0.78

Лучшей моделью оказалась LGBM именно она отправилась на тест.

Итоговое значение метрики F1 на тестовой выборке составило: 0.76